In [ ]:
#pull terms from sql ; cant concat becuase the rows not being lists will f**k you up in memory 
#name onto for matrix and output purposes 
#put each key in a dict 
    #label2ilx ... ilx2row
#parse for each key in ontologies based on accepted predicates
    #label2iri ... iri2row
# each mach 
#   ilx_iri, ilx_label, ilx_definition, ilx_synonym, ilx_superclass, ilx_type, ilx_existing_ids ... same but <Onto>Iri ... ... has_matching_label has_matching_definition ...
# 0 ...
# 1 ...

In [14]:
from collections import defaultdict
from ilxutils.ilx_pred_map import IlxPredMap
from ilxutils.interlex_sql import IlxSql
from ilxutils.mydifflib import ratio
from ilxutils.tools import open_pickle, create_pickle
import os
import pandas as pd
from pathlib import Path 
from sys import exit

ipm = IlxPredMap()
sql = IlxSql(db_url=os.environ.get('SCICRUNCH_DB_URL_PRODUCTION'))

In [2]:
%%time
cterms = sql.get_terms_complete() # final product

CPU times: user 41.2 s, sys: 4.69 s, total: 45.8 s
Wall time: 54.3 s


In [20]:
def condense_df(df: pd.DataFrame, anchor: str) -> pd.DataFrame:
    """ DataFrame of redundent data into unique rows or set
    
    Warning: anchor needs to be a singluar entity 
    
    Args: anchor: common value of which to stack the rest of the col values  
    
    Returns: pandas DataFrame where each cell is a compulated set of all the cols from which 
        have a shared anchor value
    """
    data = {}
    colnames = list(df)
    for i, row in df.iterrows(): # Cant use tuples for speed up bc of ambiguous keys
        row_anchor = str(row[anchor]) # should be a str pd object 
        for colname in colnames:
            if row[colname]:
                if not data.get(row_anchor):
                    data[row_anchor] = defaultdict(set)
                data[row_anchor][colname].add(str(row[colname]))
    records = [record for record in data.values()]
    #return pd.DataFrame(records)
    return records

#ilx = condense_df(df=cterms, anchor='ilx')

In [34]:
%%time
data = {}
for row in terms.itertuples():
    if not data.get(row.ilx):
        data[row.ilx] = defaultdict(set)
    data[row.ilx]['label'].add(row.label)
    data[row.ilx]['ilx'].add(row.ilx)
    data[row.ilx]['definition'].add(row.definition)
for row in annos.itertuples(): 
    data[row.term_ilx]['annotation_label'].add(row.annotation_label)
    data[row.term_ilx]['value'].add(row.value)
for row in synonyms.itertuples(): pass
for row in superclasses.itertuples(): pass
for row in relationships.itertuples(): pass
for row in existing_ids.itertuples(): pass

CPU times: user 2.9 s, sys: 0 ns, total: 2.9 s
Wall time: 2.9 s


In [4]:
def rm_col_nulls_repeats_type(col):
    return list(set(col[~col.isnull()]))
sub_df = cterms[cterms['ilx'] == 'ilx_0101431']
rm_col_nulls_repeats_type(sub_df['relationship'])

['ilx_0101438',
 'ilx_0381390',
 'ilx_0101999',
 'ilx_0381443',
 'ilx_0102661',
 'ilx_0101901',
 'ilx_0105004',
 'ilx_0106935',
 'ilx_0104355']

In [21]:
%time terms = sql.get_terms()

CPU times: user 7.57 s, sys: 724 ms, total: 8.29 s
Wall time: 22.8 s


In [5]:
%time annos = sql.get_annotations()

CPU times: user 29.7 s, sys: 3.84 s, total: 33.6 s
Wall time: 1min 9s


In [23]:
%time synonyms = sql.get_synonyms()

CPU times: user 2.51 s, sys: 276 ms, total: 2.78 s
Wall time: 3.25 s


In [24]:
%time superclasses = sql.get_superclasses()

CPU times: user 2.86 s, sys: 299 ms, total: 3.16 s
Wall time: 3.59 s


In [25]:
%time relationships = sql.get_relationships()

CPU times: user 130 ms, sys: 12.1 ms, total: 142 ms
Wall time: 178 ms


In [15]:
%time existing_ids = sql.get_existing_ids()

CPU times: user 15.6 s, sys: 1.74 s, total: 17.4 s
Wall time: 35.3 s


#################################################################################################################

In [10]:
terms.head()

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
0,1,ilx_0100000,32290,0,32290,0,(R)N6-phenylisopropyladenosine,,,term,6,0,1,1481226703,1529356493
1,2,ilx_0100001,32290,0,32290,0,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,,term,6,0,1,1481226705,1529356493
2,3,ilx_0100002,32290,0,34142,0,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,,term,8,0,1,1481226707,1535122210
3,4,ilx_0100003,32290,0,32290,0,1-120 Truncated Alpha-Synuclein Drosophila,This transgenic drosophila expresses a form of...,,term,6,0,1,1481226709,1529356493
4,5,ilx_0100004,32290,0,32290,0,1-D extent,A quality inhering in a bearer by virtue of it...,,term,5,0,1,1481226711,1529356493


In [6]:
annos.head()

,id,tid,annotation_tid,value,uid,term_label,term_ilx,annotation_label,annotation_ilx
0,2265164,1,304445,http://neurolex.org/wiki/nifext_5214,0,(R)N6-phenylisopropyladenosine,ilx_0100000,hasDbXref,ilx_0381360
1,2265173,2,304445,http://neurolex.org/wiki/T3D0044,0,"1,2-Dibromo-3-chloropropane",ilx_0100001,hasDbXref,ilx_0381360
2,2265170,3,304445,http://neurolex.org/wiki/T3D0035,0,"1,2-Dibromoethane",ilx_0100002,hasDbXref,ilx_0381360
3,2265171,4,304445,http://neurolex.org/wiki/nlx_organ_090801,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,hasDbXref,ilx_0381360
4,2278612,4,304535,PMID:17376994,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,literatureCitation,ilx_0381449


In [108]:
synonyms.head()

,id,tid,literal,ilx,label
0,31340,15070,Age,ilx_0115067,Participant Age
1,31341,15070,Patient Age,ilx_0115067,Participant Age
2,31342,15070,Subject Age,ilx_0115067,Participant Age
23085,331422,3729,eletriptan,ilx_0103728,Eletriptan
23086,333931,4511,GABA-A RECEPTOR,ilx_0104510,GABRA1


In [4]:
superclasses.head()

,id,tid,superclass_tid,term_label,term_ilx,superclass_label,superclass_ilx
0,12494,15070,15069,Participant Age,ilx_0115067,National Cancer Institute Common Data Element,ilx_0115066
23078,307260,2483,10842,Connected spatiotemporal region,ilx_0102482,Spatiotemporal region,ilx_0110841
23079,307261,2484,11600,Connected temporal region,ilx_0102483,Temporal region,ilx_0111599
23080,307331,2558,10743,Cord blood stem cell,ilx_0102557,Somatic stem cell,ilx_0110742
23081,307856,3092,2515,Dependent continuant,ilx_0103091,Continuant,ilx_0102514


In [7]:
relationships.head()

,id,term1_tid,term1_ilx,term1_type,term2_tid,term2_ilx,term2_type,relationship_tid,relationship_ilx,relationship_label
0,1,2,ilx_0100001,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
1,2,3,ilx_0100002,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
2,3,22,ilx_0100021,term,3721,ilx_0103720,term,12785,ilx_0112784,Has role
3,4,25,ilx_0100024,term,3577,ilx_0103576,term,12785,ilx_0112784,Has role
4,5,27,ilx_0100026,term,6459,ilx_0106458,term,12797,ilx_0112796,Related to


In [109]:
existing_ids.head()

,id,tid,curie,iri,preferred,ilx,type,label,definition,comment
0,2181964,1,NIFEXT:5214,http://uri.neuinfo.org/nif/nifstd/nifext_5214,1,ilx_0100000,term,(R)N6-phenylisopropyladenosine,,
1,2181981,1,ILX:0100000,http://uri.interlex.org/base/ilx_0100000,0,ilx_0100000,term,(R)N6-phenylisopropyladenosine,,
2,2182003,2,T3D:0044,http://t3db.org/toxins/T3D0044,0,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,
3,2182004,2,ILX:0100001,http://uri.interlex.org/base/ilx_0100001,1,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,
4,2744315,3,T3D:0035,http://t3db.org/toxins/T3D0035,0,ilx_0100002,term,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,


#################################################################################################################

In [8]:
annos[annos['value'].str.contains('PMID')]

,id,tid,annotation_tid,value,uid,term_label,term_ilx,annotation_label,annotation_ilx
4,2278612,4,304535,PMID:17376994,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,literatureCitation,ilx_0381449
92,2279520,91,304535,PMID:7580124,0,Brodmann (1909) area 46,ilx_0100090,literatureCitation,ilx_0381449
155,2278609,153,304535,PMID:17683129,0,A30P Alpha-Synuclein Drosophila,ilx_0100152,literatureCitation,ilx_0381449
159,2278611,156,304535,PMID:11590151,0,A53T Alpha-Synuclein Drosophila,ilx_0100155,literatureCitation,ilx_0381449
519,2278692,507,304535,PMID:7938162,0,Alpha2A receptor,ilx_0100506,literatureCitation,ilx_0381449
521,2278693,508,304535,PMID:7938162,0,Alpha2B receptor,ilx_0100507,literatureCitation,ilx_0381449
523,2278695,509,304535,PMID:7938162,0,Alpha2C receptor,ilx_0100508,literatureCitation,ilx_0381449
525,2278760,510,304535,PMID:7938162,0,Alpha2D receptor,ilx_0100509,literatureCitation,ilx_0381449
539,2278734,523,304535,PMID:18551525,0,Alveus,ilx_0100522,literatureCitation,ilx_0381449
540,2278737,523,304535,PMID:10906718,0,Alveus,ilx_0100522,literatureCitation,ilx_0381449


In [9]:
from ilxutils.simple_rdflib import SimpleGraph, URIRef, Literal
sg = SimpleGraph()

for i, row in annos[annos['value'].str.contains('PMID')].iterrows():
    pred = 'ilxtr:literatureCitation'
    obj = Literal(row['value'])
    sg.add_annotation(row.ilx.replace('_', ':'), )

SyntaxError: unexpected EOF while parsing (<ipython-input-9-e23884f85878>, line 4)

In [9]:
df1 = superclasses.rename({'term_ilx': 'ilx'}, axis='columns')[['ilx', 'superclass_ilx']]
df2 = existing_ids.rename({'curie': 'superclasse_curie', 'iri': 'superclass_iri'}, axis='columns')[['ilx', 'superclasse_curie', 'superclass_iri']]
direct_superclasses = pd.merge(df1, df2, on='ilx', how='outer')

In [11]:
direct_superclasses[direct_superclasses['ilx'] == 'ilx_0115067']

,ilx,superclass_ilx,superclasse_curie,superclass_iri
0,ilx_0115067,ilx_0115066,ILX:0115067,http://uri.interlex.org/base/ilx_0115067


In [76]:
sub_annotations = annos.rename({'term_ilx': 'ilx'}, axis='columns')[['ilx', 'annotation_label', 'value']]
sub_annotations.head()

,ilx,annotation_label,value
0,ilx_0100000,hasDbXref,http://neurolex.org/wiki/nifext_5214
1,ilx_0100001,hasDbXref,http://neurolex.org/wiki/T3D0044
2,ilx_0100002,hasDbXref,http://neurolex.org/wiki/T3D0035
3,ilx_0100003,hasDbXref,http://neurolex.org/wiki/nlx_organ_090801
4,ilx_0100003,literatureCitation,PMID:17376994


In [4]:
from functools import reduce
dfs = [
    terms[['id', 'ilx', 'label']],
    superclasses[['ilx', 'superclass_tid']],
    annos.rename({'term_ilx': 'ilx'}, axis='columns')[['ilx', 'annotation_label', 'value']],
    synonyms.rename({'literal': 'synonym'}, axis='columns')[['ilx', 'synonym']],
    existing_ids[['ilx', 'curie', 'iri']],
    relationships.rename({'relationship_ilx': 'ilx'}, axis='columns')[['ilx', 'term1_ilx', 'term2_ilx']],
    relationships.rename({'term1_ilx': 'ilx', 'term2_ilx': 'relationship'}, axis='columns')[['ilx', 'relationship']],
    relationships.rename({'term2_ilx': 'ilx', 'term1_ilx': 'relationship'}, axis='columns')[['ilx', 'relationship']],
]
df_final = reduce(lambda left,right: pd.merge(left,right, on='ilx', how='outer'), dfs)

NameError: name 'terms' is not defined

In [41]:
from sqlalchemy.orm.session import sessionmaker
Session = sessionmaker()
engine = create_engine(db_url)
Session.configure(bind=engine)
session = Session()
sql = f"""
    SELECT
        tei.id, tei.tid, tei.curie, tei.iri, tei.preferred,
        t.ilx, t.type, t.label, t.definition, t.comment
    FROM (
        SELECT * 
        FROM terms 
        GROUP BY terms.ilx
    ) as t
    JOIN term_existing_ids AS tei 
    ON t.id = tei.tid
""" # groub by will just take the first occurance and drop the rest
sqlobj = session.execute(sql)

In [58]:
import csv

with open('/home/tmsincomb/Dropbox/PMID/pmid-dump.tsv', 'r') as csvFile:
    reader = csv.reader(csvFile, delimiter='\t')
    old_text = []
    for i, row in enumerate(reader):
        if i == 0: 
            header = {colname: col_indx for col_indx, colname in enumerate(row)}
            continue
        old_text.append(row[header['old_text']])
    csvFile.close()
old_text[0]

'{{\\n  #if: {{{inventor-surname|{{{inventor1-surname|{{{inventor-last|{{{inventor1-last|{{{inventor|}}}}}}}}}}}}}}}\\n<!--\\n    CITATIONS FOR PATENTS\\n-->\\n|{{Citation/patent\\n |Surname1 = {{{inventor-surname|{{{inventor1-surname|{{{inventor-last|{{{inventor1-last|{{{inventor}}}}}}}}}}}}}}}\\n |Surname2={{{inventor2-surname|{{{inventor2-last|{{{inventor2|}}}}}}}}}\\n |Surname3={{{inventor3-surname|{{{inventor3-last|{{{inventor3|}}}}}}}}}\\n |Surname4={{{inventor4-surname|{{{inventor4-last|{{{inventor4|}}}}}}}}}\\n |Given1 = {{{inventor-given|{{{inventor1-given|{{{inventor-first|{{{inventor1-first|}}}}}}}}}}}}\\n |Given2={{{inventor2-given|{{{inventor2-first|}}}}}}\\n |Given3={{{inventor3-given|{{{inventor3-first|}}}}}}\\n |Given4={{{inventor4-given|{{{inventor4-first|}}}}}}\\n |Inventorlink1={{{inventorlink1|{{{inventorlink|}}}}}}\\n |Inventorlink2={{{inventorlink2|}}}\\n |Inventorlink3={{{inventorlink3|}}}\\n |Inventorlink4={{{inventorlink4|}}}\\n |Title={{{title|}}}\\n |CountryC

In [24]:
from sqlalchemy import create_engine, inspect, Table, Column
import os 
db_url = os.environ.get('SCICRUNCH_DB_URL_PRODUCTION')

def get_existing_ids():
    #return open_pickle(Path.home()/'Dropbox/interlex_backups/ilx_db_ex_backup.pickle') # DEBUG
    engine = create_engine(db_url)
    data = f"""
        SELECT
            tei.id, tei.tid, tei.curie, tei.iri, tei.preferred,
            t.ilx, t.type, t.label, t.definition, t.comment
        FROM (
            SELECT * 
            FROM terms 
            GROUP BY terms.ilx
        ) as t
        JOIN term_existing_ids AS tei 
        ON t.id = tei.tid
    """ # groub by will just take the first occurance and drop the rest
    df = pd.read_sql(data, engine)
    #df = remove_duplicates(df) # takes out dead terms
    return df
ex2 = get_existing_ids()

In [25]:
len(ex2)

566212

In [26]:
len(existing_ids)

566212

In [10]:
df1 = relationships.rename({'term1_ilx': 'ilx', 'term2_ilx': 'relationship'}, axis='columns')[['ilx', 'relationship']]
df2 = relationships.rename({'term2_ilx': 'ilx', 'term1_ilx': 'relationship'}, axis='columns')[['ilx', 'relationship']]
sub_relationship = pd.concat([df1,df2]).drop_duplicates().reset_index(drop=True)

,ilx,relationship
0,ilx_0100001,ilx_0111840
1,ilx_0100002,ilx_0111840
2,ilx_0100021,ilx_0103720
3,ilx_0100024,ilx_0103576
4,ilx_0100026,ilx_0106458
5,ilx_0100031,ilx_0111264
6,ilx_0100032,ilx_0107547
7,ilx_0100032,ilx_0110555
8,ilx_0100032,ilx_0110557
9,ilx_0100033,ilx_0100032


In [12]:
ilx = open_pickle('~/Dropbox/ilx-df.pickle')

In [13]:
ilx.head()

,annotation,definition,existing_ids,ilx,label,relationship,superclass_existing_ids,superclass_ilx,synonym
0,"{(hasDbXref, http://neurolex.org/wiki/nifext_5...",{},"{(NIFEXT:5214, http://uri.neuinfo.org/nif/nifs...",{ilx_0100000},{(R)N6-phenylisopropyladenosine},NaN,"{(ILX:0100317, http://uri.interlex.org/base/il...",{ilx_0100317},NaN
1,"{(hasDbXref, http://neurolex.org/wiki/T3D0044)}",{Organic Compound;Pesticide;Organochloride;Org...,"{(ILX:0100001, http://uri.interlex.org/base/il...",{ilx_0100001},"{1,2-Dibromo-3-chloropropane}","{(ilx_0100001, ilx_0112784, ilx_0111840)}","{(ILX:0107064, http://uri.interlex.org/base/il...",{ilx_0107064},"{Gro-tone nematode granular, C3H5Br2Cl, 1,2-Di..."
2,"{(hasDbXref, http://neurolex.org/wiki/T3D0035)}",{Organic Compound;Pesticide;Gasoline Additive/...,"{(ILX:0100002, http://uri.interlex.org/base/il...",{ilx_0100002},"{1,2-Dibromoethane}","{(ilx_0100002, ilx_0112784, ilx_0111840)}","{(ILX:0107064, http://uri.interlex.org/base/il...",{ilx_0107064},"{Soilbrome-85, 1,2-Dibromomethane, CH2BrCH2Br,..."
3,"{(literatureCitation, PMID:17376994), (hasDbXr...",{This transgenic drosophila expresses a form o...,"{(ILX:0100003, http://uri.interlex.org/base/il...",{ilx_0100003},{1-120 Truncated Alpha-Synuclein Drosophila},NaN,NaN,NaN,NaN
4,"{(hasDbXref, http://neurolex.org/wiki/PATO_000...",{A quality inhering in a bearer by virtue of i...,"{(ILX:0100004, http://uri.interlex.org/base/il...",{ilx_0100004},{1-D extent},NaN,"{(PATO:0000117, http://purl.obolibrary.org/obo...",{ilx_0110654},{1-D size}


In [126]:

df_final[df_final['ilx'] == 'ilx_0100003']

,id,ilx,label,superclass_tid,annotation_label,value,synonym,curie,iri,term1_ilx,term2_ilx,relationship
218,4,ilx_0100003,1-120 Truncated Alpha-Synuclein Drosophila,NaN,hasDbXref,http://neurolex.org/wiki/nlx_organ_090801,NaN,NLXORG:090801,http://uri.neuinfo.org/nif/nifstd/nlx_organ_09...,NaN,NaN,NaN
219,4,ilx_0100003,1-120 Truncated Alpha-Synuclein Drosophila,NaN,hasDbXref,http://neurolex.org/wiki/nlx_organ_090801,NaN,ILX:0100003,http://uri.interlex.org/base/ilx_0100003,NaN,NaN,NaN
220,4,ilx_0100003,1-120 Truncated Alpha-Synuclein Drosophila,NaN,literatureCitation,PMID:17376994,NaN,NLXORG:090801,http://uri.neuinfo.org/nif/nifstd/nlx_organ_09...,NaN,NaN,NaN
221,4,ilx_0100003,1-120 Truncated Alpha-Synuclein Drosophila,NaN,literatureCitation,PMID:17376994,NaN,ILX:0100003,http://uri.interlex.org/base/ilx_0100003,NaN,NaN,NaN


In [88]:
df_final[df_final['ilx'] == 'ilx_0100003']

,id,ilx,label,superclass_tid,annotation_label,value
